In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import json
from pandas import json_normalize
# pd.__version__

In [2]:
topic_file = '/Users/bew/Downloads/DataForThai_jupyter/DataForThai_Pandas&SubTopic/SubTopic_dataforthai.csv'
topic_table = pd.read_csv(topic_file)


topic_table['รหัส'] = topic_table['รหัส'].astype(str)
# Function to add '0' to strings of length 4
def add_zero(s):
    if len(s) == 4:
        return '0' + s
    else:
        return s
# Applying the function to the DataFrame
topic_table['รหัส'] = topic_table['รหัส'].apply(lambda x: add_zero(x))


# topic_table.dtypes
topic_table = topic_table[:91] # 3212 # update index (ไม่จำเป็นต้อง Up แค่ไม่ให้ Run เกินต้องการ)
# topic_table = topic_table[topic_table['รหัส']=='01450']
# print(topic_table)
# max(topic_table['จำนวนหน้า'])
topic_table

,รหัส,ประเภทธุรกิจ,จำนวน,จำนวนหน้า
0,01111,การปลูกข้าวโพดที่ใช้เมล็ดแก่,24,1
1,01112,การปลูกธัญพืช (ยกเว้นข้าวและข้าวโพด),34,1
2,01113,การปลูกพืชตระกูลถั่ว,15,1
3,01114,การปลูกถั่วเหลือง,3,1
4,01115,การปลูกพืชน้ำมัน (ยกเว้นถั่วเหลือง),10,1
...,...,...,...,...
86,03121,การจับปลาน้ำจืด,11,1
87,03122,การจับกุ้งน้ำจืด,1,1
88,03129,การประมงน้ำจืดอื่นๆ ซึ่งมิได้จัดประเภทไว้ในที่...,11,1
89,03211,การเพาะเลี้ยงปลาทะเล,9,1


In [3]:
# topic_for_file = topic_table[topic_table['รหัส'] == no_topic]
# topic_for_file

In [4]:
# # Read JSON file
# file_path = f'/Users/bew/Downloads/WebScrapperFor_DataForThai/Target/01111.json'  # Replace with your file path

# df = pd.read_json(file_path)

# # # Load JSON data into DataFrame
# # df = json_normalize(json_data)  # Use pd.DataFrame(json_data) if the JSON is already flat

# # df['สถานะ'] = df['สถานะ'].astype("string")
# df.dtypes
# # df

In [5]:
import pandas as pd
import os

# Initialize an empty DataFrame for the result
result = pd.DataFrame()

lastId = "03212" # update 'lastID' เพื่อไม่อ่านข้อมูลเดิม เพราะค้นหาข้อมูลมาทับ
value = ""

for index, row in topic_table.iterrows():
    
    value1 = row['รหัส']
    value2 = row['ประเภทธุรกิจ']
    
    if (int(value1) <= int(lastId)): continue
    
    # Construct the file path
    file_path = f'/Users/bew/Downloads/WebScrapperFor_DataForThai/Target/{value1}.json'
    
    # Check if the file exists
    if os.path.exists(file_path):
        try:
            # Read JSON file
            df = pd.read_json(file_path)
            
            # Add a new column with the business type
#             df['ประกอบธุรกิจ'] = value2
            
            # Concatenate the current DataFrame to the result DataFrame
            result = pd.concat([result, df], axis=0)
        
        except ValueError as e:
            print(f"Error reading JSON file at {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")
    value = value1

print(f"lastId = {value1}")

lastId = 03212


In [6]:
# Reset the index of the final result DataFrame
result = result.reset_index(drop=True)

# result

In [7]:
result.columns.tolist()

result['เบอร์โทร'] = np.nan
# new_order = ['เลขทะเบียน', 'ชื่อบริษัทภาษาไทย', 'ชื่อบริษัทภาษาอังกฤษ', 'ประกอบธุรกิจ', 'หมวดธุรกิจ', 'สถานะ',
#              'วันที่จดทะเบียน', 'ทุนจดทะเบียน', 'ที่ตั้ง', 'ข้อมูลสำหรับการติดต่อ', 'เบอร์โทร', 'Website', 'ก่อตั้งโดย',
#              'หลักทรัพย์', 'ที่มา', 'สมัครงาน / ดูตำแหน่งงานว่าง', 'วันที่ร้าง', 'วันที่เลิก']
# result = result[new_order]
# result

In [8]:
# for index, row in topic_table.iterrows():
    
#     value1 = row['รหัส']
#     value2 = row['ประเภทธุรกิจ']
    
#     # Read JSON file
#     file_path = f'/Users/bew/Downloads/WebScrapperFor_DataForThai/Target/{value1}.json'  # Replace with your file path

#     df = pd.read_json(file_path)
    
#     df['ประกอบธุรกิจ'] = value2
    
#     result = pd.concat([result, df], axis=0)
    
    
# result


In [9]:
# result = result.drop(columns=['สมัครงาน / ดูตำแหน่งงานว่าง', 'วันที่ร้าง', 'หลักทรัพย์'])

result = result.applymap(str)
# result['เลขทะเบียน'] = result['เลขทะเบียน'].map(str)
# result['สถานะ'] = result['สถานะ'].astype("string")


# Step 1: Remove commas from the strings
result['ทุนจดทะเบียน'] = result['ทุนจดทะเบียน'].str.replace(',', '')
result['ทุนจดทะเบียน'] = result['ทุนจดทะเบียน'].str.replace(' บาท', '')
# Step 2: Convert the strings to numeric type (int or float)
result['ทุนจดทะเบียน'] = pd.to_numeric(result['ทุนจดทะเบียน'])

result.dtypes

เลขทะเบียน                     object
ประกอบธุรกิจ                   object
หมวดธุรกิจ                     object
สถานะ                          object
วันที่จดทะเบียน                object
ทุนจดทะเบียน                    int64
ก่อตั้งโดย                     object
ที่ตั้ง                        object
ชื่อบริษัทภาษาอังกฤษ           object
ชื่อบริษัทภาษาไทย              object
ข้อมูลสำหรับการติดต่อ          object
ที่มา                          object
วันที่เลิก                     object
เว็บไซต์                       object
สมัครงาน / ดูตำแหน่งงานว่าง    object
เบอร์โทร                       object
dtype: object

In [10]:
result = result[result['สถานะ']=='ยังดำเนินกิจการอยู่']
result = result[(result['ทุนจดทะเบียน'] >= 30000000)]

# def get_length(obj):
#     try:
#         return len(obj)
#     except TypeError:
#         return None

# for text in result['ทุนจดทะเบียน']:
#     print(text.apply(get_length))

# for index, row in result.iterrows():
    
#     value1 = row['เลขทะเบียน']
#     if value1.apply(len) != 13:
#         value1 = "0" + value1.astype(str)

result['เลขทะเบียน'] = "'0" + result['เลขทะเบียน'].astype(str)
# # Function to add '0' to strings of length 13
# def add_zero(s):
# #     s = str(s)  # Ensure the input is a string
#     if len(s) == 14:
#         return '0' + s
#     else:
#         return s

# # Applying the function to the DataFrame
# result['เลขทะเบียน'] = result['เลขทะเบียน'].apply(lambda x: add_zero(x))

# # Reset the index of the final result DataFrame
# result = result.reset_index(drop=True)

# result.dtypes

In [11]:
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' มกราคม ','/1/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' กุมภาพันธ์ ','/2/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' มีนาคม ','/3/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' เมษายน ','/4/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' พฤษภาคม ','/5/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' มิถุนายน ','/6/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' กรกฎาคม ','/7/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' สิงหาคม ','/8/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' กันยายน ','/9/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' ตุลาคม ','/10/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' พฤศจิกายน ','/11/')
result['วันที่จดทะเบียน'] = result['วันที่จดทะเบียน'].str.replace(' ธันวาคม ','/12/')

In [12]:
result = result.reset_index(drop=True)
# result

In [13]:
# result.dtypes

In [14]:
file = '/Users/bew/Downloads/DataForThai_jupyter/FilterAndSearchData.csv'
df = pd.read_csv(file)

df_updated = pd.concat([df, result], ignore_index=True)
df_updated = df_updated.drop_duplicates(subset='เลขทะเบียน')
df_updated = df_updated.reset_index(drop=True)

df_updated.dtypes
df_updated

,เลขทะเบียน,ประกอบธุรกิจ,หมวดธุรกิจ,สถานะ,วันที่จดทะเบียน,ทุนจดทะเบียน,Website,ก่อตั้งโดย,ที่ตั้ง,ชื่อบริษัทภาษาอังกฤษ,...,ข้อมูลสำหรับการติดต่อ,ที่มา,สมัครงาน / ดูตำแหน่งงานว่าง,วันที่ร้าง,หลักทรัพย์,วันที่เลิก,เบอร์โทร,ข้อมูลสำหรับติดต่อ,เสร็จชำระบัญชี,เว็บไซต์
0,'0105523007276,ผลิตเมล็ดพันธุ์ข้าวโพดและข้าวฟ่าง จำหน่ายเมล็ด...,การปลูกข้าวโพดที่ใช้เมล็ดแก่,ยังดำเนินกิจการอยู่,29/4/2523,245000000,NaN,"กรรมการ: นายกิตติพงษ์ ลิมปสุรัติ, นางจริยา วุฒ...",87 อาคารเอ็ม.ไทย ทาวเวอร์ ออลซีซันส์ เพลส ชั้น...,"PIONEER HI-BRED (THAILAND) CO., LTD.",...,False,https://www.dataforthai.com/company/0105523007...,ดูตำแหน่งงานจาก JOBTOPGUN,NaN,NaN,NaN,'027922900,NaN,NaN,NaN
1,'0105539070673,จำหน่ายเมล็ดพันธุ์,การปลูกธัญพืช (ยกเว้นข้าวและข้าวโพด),ยังดำเนินกิจการอยู่,26/6/2539,162000000,NaN,"กรรมการ: นายโยชิฮิโร โอบาทาเกะ, นายเซน ทากามิย...",260 หมู่ที่ 13 ตำบลโนนท่อน อำเภอเมืองขอนแก่น จ...,"SAKATA SIAM SEED CO., LTD.",...,False,https://www.dataforthai.com/company/0105539070...,NaN,NaN,NaN,NaN,'043001314,NaN,NaN,NaN
2,'0105563169226,เพาะปลูกผลิตผลทางการเกษตรกรรม,การปลูกข้าวจ้าว,ยังดำเนินกิจการอยู่,17/11/2563,100000000,NaN,กรรมการ: นางสาวอรวรรณ พัฒนพล,71 อาคารเอจีอาร์ ชั้นที่ 4 ถนนเจริญราษฎร์ แขวง...,SUBNAIDIN 666 COMPANY LIMITED,...,False,https://www.dataforthai.com/company/0105563169...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,'0815524000023,ประกอบกิจการเกษตรกรรม,การปลูกธัญพืช (ยกเว้นข้าวและข้าวโพด),ยังดำเนินกิจการอยู่,17/9/2524,30000000,NaN,"กรรมการ: นายบุญส่ง ลีลามั่นคง, นางสุรัตน์ โพธิ...",141 หมู่ที่ 2 ถนนเพชรเกษม ตำบลอ่าวลึกเหนือ อำเ...,NaN,...,False,https://www.dataforthai.com/company/0815524000...,NaN,NaN,NaN,NaN,'075681136,NaN,NaN,NaN
4,'0105531018826,การเกษตรกรรมทุกชนิด และการให้บริการ,การปลูกธัญพืช (ยกเว้นข้าวและข้าวโพด),ยังดำเนินกิจการอยู่,16/3/2531,150000000,NaN,"กรรมการ: นางสาววรรณี จันทร์ไกรวัล, นางณัฐนี ตุ...",109 หมู่ที่ 9 ถนนสุวรรณศร ตำบลดงขี้เหล็ก อำเภอ...,"MORI FOOD PRODUCTS CO., LTD.",...,False,https://www.dataforthai.com/company/0105531018...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,'0845546002592,เพาะเลี้ยงกุ้งทะเล,การเพาะเลี้ยงกุ้งทะเล,ยังดำเนินกิจการอยู่,10/7/2546,300000000,NaN,"กรรมการ: นายแทน เทือกสุบรรณ, นางสาวแขแสง เทือก...",73/6-7 หมู่ที่ 2 ตำบลพุนพิน อำเภอพุนพิน จ.สุรา...,SRISUBAN FARM COMPANY LIMITED,...,False,https://www.dataforthai.com/company/0845546002...,nan,NaN,NaN,nan,nan,NaN,NaN,http://www.srisubanfarm.com
668,'0915532000010,ฟาร์มเลี้ยงกุ้ง,การเพาะเลี้ยงกุ้งทะเล,ยังดำเนินกิจการอยู่,5/1/2532,50000000,NaN,กรรมการ: นางทิพรัตน์ รัชกิจประการ,31/2 ถนนสตูลธานี ตำบลพิมาน อำเภอเมืองสตูล จ.สต...,,...,False,https://www.dataforthai.com/company/0915532000...,nan,NaN,NaN,nan,nan,NaN,NaN,nan
669,'0105535143170,ดำเนินธุรกิจเกี่ยวกับทำฟาร์มกุ้ง และจำหน่ายอาห...,การเพาะเลี้ยงกุ้งทะเล,ยังดำเนินกิจการอยู่,20/10/2535,32000000,NaN,กรรมการ: นางสาวเทล่า เอลี่,28/1 หมู่ที่ 5 ตำบลคลองใหญ่ อำเภอแหลมงอบ จ.ตรา...,"SIAM MATURAL RESOURCES CO., LTD.",...,False,https://www.dataforthai.com/company/0105535143...,nan,NaN,NaN,nan,nan,NaN,NaN,nan
670,'0765556000712,เพาะเลี้ยงและจำหน่ายสัตว์น้ำ,การเพาะเลี้ยงสัตว์น้ำจืดอื่นๆ ซึ่งมิได้จัดประเ...,ยังดำเนินกิจการอยู่,4/9/2556,55500000,NaN,"กรรมการ: นางนิชาภา สืบทิมรัตน์, นางคาโยะ มัตทส...",123/15 หมู่ที่ 3 ตำบลเขาแดง อำเภอกุยบุรี จ.ประ...,S.I.A. FARM COMPANY LIMITED,...,False,https://www.dataforthai.com/company/0765556000...,nan,NaN,NaN,nan,nan,NaN,NaN,nan


In [15]:
# result[result['เลขทะเบียน'].str.len() != 13]

In [16]:
# result

In [19]:
file_name = 'FilterAndSearchData.csv'
file_path = '/Users/bew/Downloads/DataForThai_jupyter/'

# Combine the file path and file name
full_path = os.path.join(file_path, file_name)

# result.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8') # รอบ 1
df_updated.to_csv(full_path, sep=',', index=False, header=True, encoding='utf-8') # รอบ 2+

print('finish')

finish


In [18]:
# # Step 1: Remove commas from the strings
# df['Capital'] = df['Capital'].str.replace(',', '')

# # Step 2: Convert the strings to numeric type (int or float)
# df['Capital'] = pd.to_numeric(df['Capital'])

# # Display the DataFrame to verify the changes
# df